In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("./cmd/addr.txt",header=None,sep='\t',dtype=str,names=['type','code','name'])

In [8]:
province = df[df['type']=='province']

In [21]:
def process(row):
    name = row['name']
    if name.endswith('市'):
        return [name, name.rstrip('市')]
    elif name.endswith('省'):
        return [name, name.rstrip('省')]
    elif name == '内蒙古自治区':
        return [name, '内蒙古省', '内蒙古', '内蒙']
    elif name == '广西壮族自治区':
        return [name, '广西自治区', '广西省', '广西']
    elif name == '西藏自治区':
        return [name, '西藏省', '西藏']
    elif name == '新疆维吾尔自治区':
        return [name,'新疆维吾尔族自治区', '新疆自治区', '新疆省', '新疆']
    return [name]

In [26]:
#province['ext'] = province.apply(process, axis=1)

In [27]:
city = df[df['type']=='city']

In [82]:
zu = open("minzu.txt").read().strip().split('\n')
def _zhou(name):
    x = name.rstrip('自治州')
    for i in zu:
        x = x.replace(i,"")
    ret = [name,x+"自治州",x+"州",x+"市",x]
    if name == '克孜勒苏柯尔克孜自治州': ret.append("克州")
    elif name == '巴音郭楞蒙古自治州': ret.append("巴州")
    elif name == '博尔塔拉蒙古自治州': ret.append("博州")
    return ret
def city_process(row):
    name = row['name']
    if name.endswith('市'):
        ret = [name, name.rstrip('市')]
        if name == '呼和浩特市': ret.append('呼市')
        return ret
    elif name.endswith('盟'):
        return [name, name.rstrip('盟')]
    elif name.endswith("自治州"):
        return _zhou(name)
    elif name.endswith("地区"):
        return [name, name.rstrip("地区")]
    return [name]

In [140]:
#city[~city.name.str.match('.*(市|盟|区|区划|县)$')].name.apply(_zhou)
#city.apply(city_process,axis=1)
#city[~city.name.str.match('.*(市|盟|自治州|地区)$')]

In [87]:
county = df[df['type']=='county']

In [131]:
county_suffix = ['新区', '矿区', '区', '县', '市', '现代产业园', '行政委员会', '管委会']
def county_process(row):
    name = row['name']
    if len(name) <= 2: return [name] # 赵县
    for suffix in county_suffix:
        if name.endswith(suffix):
            return [name, name[:-len(suffix)]]
    if name.endswith('自治旗'):
        x = name[:-len('自治旗')]
        return [name, x+'旗', x]
    elif name.endswith('旗'):
        return [name, name.rstrip('旗')]
    elif name == '保定白沟新城':
        return [name, '白沟']
    elif name.endswith('群岛'):
        return [name, name.rstrip('群岛')]
    elif name == '中沙群岛的岛礁及其海域':
        return [name, '中沙群岛', '中沙']
    return [name]

In [141]:
#county.apply(county_process,axis=1)
#county[~county.name.str.match('.*(县|区|市|现代产业园|行政委员会|管委会|自治旗|旗|保定白沟新城|群岛)$')]

In [112]:
town = df[df['type']=='town']

In [116]:
town.name.to_csv('town.txt',index=None,header=None)

In [117]:
!goname -i town.txt

 43486 / 43486 [====================================================] 100.00% 0s
done!


In [135]:
town_suffix = ['镇', '乡', '街道办事处', '街道', '地区办事处']
def town_process(row):
    name = row['name']
    for suffix in town_suffix:
        if name.endswith(suffix):
            return [name, name[:-len(suffix)]]
    return [name]

In [139]:
#town.apply(town_process,axis=1)
#town[~town.name.str.match('.*(镇|乡|街道办事处|地区办事处|街道)$')]

In [142]:
village = df[df['type']=='village']

In [143]:
village.name.to_csv('village.txt',index=None,header=None)

In [144]:
!goname -i village.txt -o village_suffix.txt

 270546 / 270546 [==================================================] 100.00% 8s
done!


In [174]:
def village_process(row):
    name = row['name']
    ss = ["村委会","村民委员会","社区村委会","嘎查","嘎查村","嘎查村委员会"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            if x.endswith('村'):
                return [name, x, x.rstrip('村')]
            else:
                return [name, x+'村', x]
    
    ss = ["社区居委会","社区居民委员会","居民委员会","居委会","社区","社区委员会","社区居委","社区居民委会"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            return [name, x+'社区', x+"小区", x]

    ss = ["生活区","工作区","工业园","工业区"]
    for s in ss:
        if name.endswith(s):
            x = name[:-len(s)]
            return [name, x]
    return [name]

In [178]:
#village[village.name.str.match(".*(嘎查)$")].apply(village_process, axis=1)
ss = ["村委会","村民委员会","社区村委会","嘎查","嘎查村","嘎查村委员会"]+["社区居委会","社区居民委员会"]+["社区"]
ss += ["居民委员会","居委会","社区","社区委员会","社区居委","生活区","工作区","工业园","工业区","社区居民委会"]
village[~village.name.str.match(".*("+'|'.join(ss)+")$")].head()

,type,code,name
52121,village,110115004402,亦庄镇南部多功能配套区
52236,village,110115006402,南郊旧宫场区
52439,village,110115103401,青云店镇工业开发区
52498,village,110115104401,北京采育经济开发区
52695,village,110115108401,庞各庄镇开发区
